In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as mtr

dataframe = pd.read_csv("../SerieA/Season21_22/matches.csv", index_col=0)
#matches["venue_code"] = matches["venue"].astype("category").cat.code

In [2]:
home_codes = pd.Categorical(dataframe.h_team).codes
away_codes = pd.Categorical(dataframe.a_team).codes

dataframe['h_code'] = home_codes
dataframe['a_code'] = away_codes
#dataframe.to_csv("temp.csv")
dataframe.columns


Index(['id', 'h_team', 'h_goals', 'a_team', 'a_goals', 'result', 'date',
       'matchday', 'h_total_shots', 'h_shots_on_target', 'a_total_shots',
       'a_shots_on_target', 'h_goals_on_penalty', 'h_total_penalties',
       'a_goals_on_penalty', 'a_total_penalties', 'h_completed_passings',
       'h_total_passings', 'a_completed_passings', 'a_total_passings',
       'h_corners', 'a_corners', 'h_fouls', 'h_yellow_cards', 'h_red_cards',
       'a_yellow_cards', 'a_red_cards', 'a_fouls', 'h_percentage_possession',
       'a_percentage_possession', 'h_code', 'a_code'],
      dtype='object')

In [3]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=10000), #max_iter di default vale 100, ho dovuto alzarlo se no non converge
    'Support Vector Machine': SVC(),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

In [4]:
predictors = ["h_code", "a_code"]
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(dataframe[predictors], dataframe.result, test_size=0.2, random_state=42)

In [5]:
predictions = {}
for model_name, model in models.items():
    models[model_name].fit(X_train, y_train)
    predictions[model_name] = model.predict(X_test)

In [6]:
E = []
for estimator, y_pred in predictions.items():
    report = mtr.classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    E.append({
        'Model': estimator, 'Accuracy': report['accuracy'],
        'Avg Precision (macro)': report['macro avg']['precision'],
        'Avg Recall (macro)': report['macro avg']['recall'],
        'Avg F1-score (macro)': report['macro avg']['f1-score'],
        'Avg Precision (weighted)': report['weighted avg']['precision'],
        'Avg Recall (weighted)': report['weighted avg']['recall'],
        'Avg F1-score (weighted)': report['weighted avg']['f1-score']
    })
E = pd.DataFrame(E).set_index('Model', inplace=False)

In [7]:
E

,Accuracy,Avg Precision (macro),Avg Recall (macro),Avg F1-score (macro),Avg Precision (weighted),Avg Recall (weighted),Avg F1-score (weighted)
Model,,,,,,,
Logistic Regression,0.355263,0.251948,0.313131,0.257453,0.302358,0.355263,0.300706
Support Vector Machine,0.552632,0.380952,0.478114,0.410055,0.456767,0.552632,0.483624
Multinomial Naive Bayes,0.368421,0.346867,0.340208,0.325805,0.391940,0.368421,0.358928
Decision Tree,0.342105,0.334722,0.317270,0.320597,0.377796,0.342105,0.353577
Random Forest,0.407895,0.367185,0.366021,0.365437,0.415149,0.407895,0.410135
K-Nearest Neighbors,0.368421,0.339685,0.331229,0.333958,0.386271,0.368421,0.375626


Voglio provare a prendere le partite del test set e calcolare gli expected goals

In [8]:
teams = sorted(dataframe.h_team.unique())
dataset = dataframe[:300]
for team in teams[:2]:
    tH = dataset[dataset["h_team"]==team]
    tA = dataset[dataset["a_team"]==team]
    df = tH.append(tA)
    df = df.sort_values(by=['date'])


Per poter migliorare il punteggio di accuratezza, bisogna fare delle medie tra le ultime partite. Quindi raggruppo in ogni singolo dataset tutte le partite giocate da una singola squadra

In [9]:
h_grouped_matches = dataframe.groupby("h_team")
a_grouped_matches = dataframe.groupby("a_team")

list_datasets = []

convertion_h = {'h_team': 'team',
        'h_goals': 'goals', 
        'h_total_shots' : 'total_shots', 
        'h_shots_on_target' : 'shots_on_target', 
        'h_goals_on_penalty': 'goals_on_penalty', 
        'h_total_penalties': 'total_penalties', 
        'h_completed_passings': 'completed_passings',
        'h_total_passings': 'total_passings',
        'h_corners': 'corners',
        'h_fouls': 'fouls',
        'h_yellow_cards': 'yellow_cards', 
        'h_red_cards': 'red_cards', 
        'h_percentage_possession': 'percentage_possession',
        'h_code': 'code'}

convertion_a = {'a_team': 'team',
        'a_goals': 'goals', 
        'a_total_shots' : 'total_shots', 
        'a_shots_on_target': 'shots_on_target', 
        'a_goals_on_penalty': 'goals_on_penalty', 
        'a_total_penalties': 'total_penalties',
        'a_completed_passings': 'completed_passings', 
        'a_total_passings': 'total_passings',
        'a_corners' : 'corners',
        'a_yellow_cards': 'yellow_cards', 
        'a_red_cards': 'red_cards', 
        'a_fouls': 'fouls',
        'a_percentage_possession': 'percentage_possession',
        'a_code': 'code'}
        

In [10]:
a_cols = [a_col for a_col in dataframe.columns if a_col[0] == 'a']
h_cols = [h_col for h_col in dataframe.columns if h_col[0] == 'h']

for team in sorted(dataframe.h_team.unique()):
    h_group = h_grouped_matches.get_group(team) 
    a_group = a_grouped_matches.get_group(team)
    
    h_group = h_group.drop(columns=a_cols) #elimino tutte le colonne relative alla squadra in trasferta (la squadra attuale significa che gioca in casa in quel match)
    h_group = h_group.rename(columns=convertion_h) #normalizzazione delle colonne, così se la squadra gioca in casa o in trasferta, i dati poi appaiono nelle stesse colonne perché non ci sono più i prefissi h_ o a_
    a_group = a_group.drop(columns=h_cols) #elimino tutte le colonne relative alla squadra in casa (la squadra attuale significa che gioca in trasferta in quel match)
    a_group = a_group.rename(columns=convertion_a)

    df = h_group.append(a_group)
    df = df.sort_values('date')
    df = df.drop(columns={'matchday', 'date'})
    list_datasets.append(df)

In [11]:
cols = [val for val in list_datasets[0].columns.values if (val != 'id') & (val != 'result') & (val != 'code') & (val != 'team')]
new_cols = [col+'_mean' for col in cols]

#calcolo la media per le 3 partite della stessa squadra
for df in list_datasets:
    rolling_stats = df[cols].rolling(3, closed='left').mean()
    df[new_cols] = rolling_stats
    df = df[3:]
    display(df)


,id,team,goals,result,total_shots,shots_on_target,goals_on_penalty,total_penalties,completed_passings,total_passings,...,shots_on_target_mean,goals_on_penalty_mean,total_penalties_mean,completed_passings_mean,total_passings_mean,corners_mean,fouls_mean,yellow_cards_mean,red_cards_mean,percentage_possession_mean
0,33,Atalanta,1,P,10.0,3.0,0,0,473.0,567.0,...,3.000000,0.333333,0.333333,425.000000,538.333333,5.333333,13.000000,2.666667,0.000000,55.666667
0,42,Atalanta,2,V,14.0,5.0,0,0,398.0,484.0,...,3.333333,0.333333,0.333333,473.333333,573.333333,6.333333,13.333333,3.333333,0.000000,57.000000
0,51,Atalanta,2,N,21.0,5.0,0,0,351.0,429.0,...,4.000000,0.333333,0.333333,444.666667,535.333333,6.333333,16.333333,3.333333,0.000000,53.000000
0,68,Atalanta,2,P,16.0,4.0,1,1,406.0,499.0,...,4.333333,0.000000,0.000000,407.333333,493.333333,6.666667,16.000000,3.666667,0.000000,51.333333
0,76,Atalanta,4,P,21.0,7.0,0,1,401.0,490.0,...,4.666667,0.333333,0.333333,385.000000,470.666667,8.333333,15.333333,2.666667,0.000000,51.000000
0,88,Atalanta,1,N,8.0,3.0,0,0,582.0,683.0,...,5.333333,0.333333,0.666667,386.000000,472.666667,8.000000,14.000000,2.666667,0.000000,52.666667
0,96,Atalanta,3,P,13.0,4.0,0,0,499.0,611.0,...,4.666667,0.333333,0.666667,463.000000,557.333333,6.333333,15.333333,1.666667,0.000000,58.333333
0,102,Atalanta,2,N,17.0,3.0,0,0,531.0,634.0,...,4.666667,0.000000,0.333333,494.000000,594.666667,4.333333,17.333333,2.333333,0.000000,59.666667
0,111,Atalanta,2,P,20.0,8.0,0,0,539.0,641.0,...,3.333333,0.000000,0.000000,537.333333,642.666667,5.000000,16.333333,2.000000,0.000000,61.000000
0,121,Atalanta,5,V,17.0,8.0,1,1,449.0,529.0,...,5.000000,0.000000,0.000000,523.000000,628.666667,5.333333,14.333333,1.666667,0.000000,59.333333


,id,team,goals,result,total_shots,shots_on_target,goals_on_penalty,total_penalties,completed_passings,total_passings,...,shots_on_target_mean,goals_on_penalty_mean,total_penalties_mean,completed_passings_mean,total_passings_mean,corners_mean,fouls_mean,yellow_cards_mean,red_cards_mean,percentage_possession_mean
0,31,Bologna,1,V,8.0,2.0,0,0,486.0,546.0,...,4.000000,0.000000,0.000000,346.333333,457.000000,6.666667,14.333333,4.000000,0.666667,52.000000
0,41,Bologna,2,N,16.0,4.0,1,1,471.0,567.0,...,2.333333,0.000000,0.000000,346.666667,443.000000,4.000000,14.333333,3.000000,0.000000,46.000000
0,56,Bologna,2,V,20.0,5.0,0,1,420.0,500.0,...,3.333333,0.333333,0.333333,404.666667,491.666667,4.333333,14.000000,2.333333,0.000000,52.000000
0,65,Bologna,3,V,10.0,3.0,0,0,378.0,462.0,...,3.666667,0.333333,0.666667,459.000000,537.666667,5.666667,16.000000,2.666667,0.000000,58.666667
0,78,Bologna,1,N,24.0,11.0,0,0,460.0,526.0,...,4.000000,0.333333,0.666667,423.000000,509.666667,6.333333,14.666667,2.333333,0.000000,54.333333
0,84,Bologna,2,P,9.0,4.0,0,0,305.0,382.0,...,6.333333,0.000000,0.333333,419.333333,496.000000,7.000000,13.333333,3.666667,0.000000,54.333333
0,99,Bologna,0,V,8.0,1.0,0,0,324.0,392.0,...,6.000000,0.000000,0.000000,381.000000,456.666667,5.000000,9.000000,3.333333,0.666667,46.000000
0,109,Bologna,2,V,8.0,3.0,0,0,553.0,638.0,...,5.333333,0.000000,0.000000,363.000000,433.333333,4.666667,8.666667,3.000000,0.666667,44.000000
0,116,Bologna,2,P,14.0,6.0,0,0,410.0,516.0,...,2.666667,0.000000,0.000000,394.000000,470.666667,3.666667,9.333333,1.666667,0.666667,43.333333
0,126,Bologna,0,P,24.0,6.0,0,0,603.0,711.0,...,3.333333,0.000000,0.000000,429.000000,515.333333,4.000000,11.000000,1.666667,0.000000,48.333333


,id,team,goals,result,total_shots,shots_on_target,goals_on_penalty,total_penalties,completed_passings,total_passings,...,shots_on_target_mean,goals_on_penalty_mean,total_penalties_mean,completed_passings_mean,total_passings_mean,corners_mean,fouls_mean,yellow_cards_mean,red_cards_mean,percentage_possession_mean
0,34,Cagliari,2,N,10.0,3.0,0,0,198.0,296.0,...,3.333333,0.666667,0.666667,401.000000,492.000000,5.333333,15.000000,3.000000,0.000000,51.666667
0,44,Cagliari,0,P,15.0,1.0,0,0,376.0,489.0,...,2.333333,0.333333,0.333333,330.666667,417.666667,2.000000,13.666667,3.333333,0.333333,41.333333
0,53,Cagliari,0,V,6.0,0.0,0,0,297.0,375.0,...,2.000000,0.333333,0.333333,336.333333,439.666667,3.333333,14.666667,2.333333,0.333333,43.333333
0,60,Cagliari,1,N,9.0,3.0,0,0,310.0,401.0,...,1.333333,0.000000,0.000000,290.333333,386.666667,4.000000,13.666667,2.000000,0.333333,36.666667
0,77,Cagliari,3,V,12.0,5.0,0,0,228.0,329.0,...,1.333333,0.000000,0.000000,327.666667,421.666667,4.333333,14.666667,1.666667,0.000000,41.333333
0,89,Cagliari,0,V,3.0,1.0,0,0,269.0,358.0,...,2.666667,0.000000,0.000000,278.333333,368.333333,3.666667,14.666667,2.333333,0.000000,38.000000
0,98,Cagliari,1,P,7.0,5.0,0,0,264.0,355.0,...,3.000000,0.000000,0.000000,269.000000,362.666667,2.666667,15.333333,2.666667,0.000000,38.666667
0,109,Cagliari,0,V,9.0,2.0,0,0,400.0,476.0,...,3.666667,0.000000,0.000000,253.666667,347.333333,2.666667,16.000000,3.000000,0.000000,38.666667
0,111,Cagliari,1,P,7.0,1.0,0,0,287.0,389.0,...,2.666667,0.000000,0.000000,311.000000,396.333333,1.666667,14.333333,2.333333,0.333333,40.000000
0,123,Cagliari,2,N,10.0,3.0,1,1,297.0,392.0,...,2.666667,0.000000,0.000000,317.000000,406.666667,2.000000,14.666667,2.000000,0.333333,41.000000


,id,team,goals,result,total_shots,shots_on_target,goals_on_penalty,total_penalties,completed_passings,total_passings,...,shots_on_target_mean,goals_on_penalty_mean,total_penalties_mean,completed_passings_mean,total_passings_mean,corners_mean,fouls_mean,yellow_cards_mean,red_cards_mean,percentage_possession_mean
0,35,Empoli,0,P,15.0,4.0,0,0,349.0,456.0,...,4.000000,0.333333,0.333333,400.333333,500.000000,7.333333,12.000000,2.333333,0.000000,53.666667
0,44,Empoli,2,P,18.0,5.0,0,0,394.0,499.0,...,4.000000,0.333333,0.333333,380.000000,486.666667,8.333333,10.666667,2.333333,0.000000,56.000000
0,56,Empoli,4,V,14.0,6.0,1,1,219.0,299.0,...,4.666667,0.333333,0.333333,414.000000,520.666667,6.666667,12.666667,1.666667,0.000000,57.666667
0,64,Empoli,0,V,12.0,3.0,0,0,304.0,368.0,...,5.000000,0.333333,0.333333,320.666667,418.000000,4.666667,13.666667,1.666667,0.000000,47.000000
0,76,Empoli,1,P,13.0,4.0,0,0,355.0,459.0,...,4.666667,0.333333,0.333333,305.666667,388.666667,3.666667,15.666667,2.000000,0.000000,41.333333
0,82,Empoli,4,P,9.0,4.0,1,1,268.0,371.0,...,4.333333,0.333333,0.333333,292.666667,375.333333,4.000000,14.666667,2.666667,0.000000,40.333333
0,97,Empoli,0,P,9.0,1.0,0,0,235.0,301.0,...,3.666667,0.333333,0.333333,309.000000,399.333333,4.000000,15.000000,3.000000,0.000000,41.666667
0,108,Empoli,2,P,13.0,4.0,1,1,323.0,418.0,...,3.000000,0.333333,0.333333,286.000000,377.000000,4.000000,16.666667,2.333333,0.333333,40.000000
0,110,Empoli,2,N,10.0,2.0,0,0,341.0,467.0,...,3.000000,0.666667,0.666667,275.333333,363.333333,4.333333,18.000000,2.000000,0.333333,38.333333
0,129,Empoli,1,V,9.0,3.0,0,0,278.0,401.0,...,2.333333,0.333333,0.333333,299.666667,395.333333,5.000000,15.333333,1.000000,0.333333,42.333333


,id,team,goals,result,total_shots,shots_on_target,goals_on_penalty,total_penalties,completed_passings,total_passings,...,shots_on_target_mean,goals_on_penalty_mean,total_penalties_mean,completed_passings_mean,total_passings_mean,corners_mean,fouls_mean,yellow_cards_mean,red_cards_mean,percentage_possession_mean
0,32,Fiorentina,2,P,16.0,5.0,0,0,531.0,617.0,...,4.666667,0.666667,0.666667,411.666667,492.666667,3.333333,16.333333,3.333333,0.333333,54.000000
0,40,Fiorentina,1,P,14.0,6.0,0,0,452.0,530.0,...,3.666667,0.666667,0.666667,422.666667,510.333333,5.666667,17.333333,4.000000,0.000000,55.666667
0,57,Fiorentina,1,P,7.0,1.0,1,1,518.0,599.0,...,3.666667,0.666667,0.666667,452.333333,533.666667,6.000000,17.000000,3.333333,0.333333,53.666667
0,67,Fiorentina,1,P,10.0,3.0,0,0,424.0,511.0,...,4.000000,0.333333,0.333333,500.333333,582.000000,5.333333,14.666667,3.000000,0.333333,56.666667
0,79,Fiorentina,0,V,11.0,3.0,0,0,535.0,627.0,...,3.333333,0.333333,0.333333,464.666667,546.666667,4.333333,15.000000,2.666667,0.333333,54.000000
0,89,Fiorentina,3,V,20.0,4.0,1,1,583.0,659.0,...,2.333333,0.333333,0.333333,492.333333,579.000000,4.666667,14.333333,3.666667,0.333333,56.666667
0,95,Fiorentina,0,V,9.0,1.0,0,0,485.0,573.0,...,3.333333,0.333333,0.333333,514.000000,599.000000,5.666667,13.333333,2.666667,0.333333,60.333333
0,103,Fiorentina,3,V,26.0,11.0,1,1,419.0,491.0,...,2.666667,0.333333,0.333333,534.333333,619.666667,5.000000,12.000000,2.666667,0.333333,59.000000
0,112,Fiorentina,0,V,6.0,1.0,0,0,335.0,426.0,...,5.333333,0.666667,0.666667,495.666667,574.333333,5.666667,11.666667,2.333333,0.000000,57.000000
0,120,Fiorentina,4,V,9.0,5.0,0,0,361.0,449.0,...,4.333333,0.333333,0.333333,413.000000,496.666667,6.000000,13.666667,3.666667,0.333333,52.000000


...

,id,team,goals,result,total_shots,shots_on_target,goals_on_penalty,total_penalties,completed_passings,total_passings,...,shots_on_target_mean,goals_on_penalty_mean,total_penalties_mean,completed_passings_mean,total_passings_mean,corners_mean,fouls_mean,yellow_cards_mean,red_cards_mean,percentage_possession_mean
0,30,Sassuolo,0,P,10.0,2.0,0,0,390.0,485.0,...,6.333333,0.000000,0.000000,482.666667,558.333333,6.000000,11.666667,1.666667,0.000000,58.000000
0,42,Sassuolo,1,V,8.0,1.0,0,0,431.0,542.0,...,4.666667,0.000000,0.000000,452.333333,540.000000,6.666667,9.666667,1.000000,0.000000,55.333333
0,54,Sassuolo,1,V,17.0,6.0,0,0,409.0,491.0,...,3.333333,0.000000,0.000000,419.666667,511.666667,5.666667,12.666667,1.000000,0.000000,52.000000
0,62,Sassuolo,1,P,11.0,7.0,1,1,432.0,524.0,...,3.000000,0.000000,0.000000,410.000000,506.000000,5.000000,14.666667,1.666667,0.000000,54.000000
0,75,Sassuolo,2,N,15.0,5.0,0,0,439.0,517.0,...,4.666667,0.333333,0.333333,424.000000,519.000000,5.000000,15.333333,3.000000,0.000000,55.000000
0,83,Sassuolo,3,V,14.0,7.0,0,0,584.0,654.0,...,6.000000,0.333333,0.333333,426.666667,510.666667,3.666667,16.666667,3.666667,0.000000,56.333333
0,94,Sassuolo,2,P,8.0,4.0,0,0,346.0,426.0,...,6.333333,0.333333,0.333333,485.000000,565.000000,4.000000,15.666667,3.000000,0.000000,58.333333
0,108,Sassuolo,1,P,15.0,5.0,0,0,471.0,549.0,...,5.333333,0.000000,0.000000,456.333333,532.333333,3.333333,18.000000,3.333333,0.000000,54.333333
0,114,Sassuolo,2,V,10.0,4.0,0,0,513.0,616.0,...,5.333333,0.000000,0.000000,467.000000,543.000000,4.333333,17.333333,3.333333,0.000000,54.333333
0,123,Sassuolo,2,N,19.0,3.0,1,1,469.0,545.0,...,4.333333,0.000000,0.000000,443.333333,530.333333,4.333333,17.666667,4.000000,0.000000,54.000000


,id,team,goals,result,total_shots,shots_on_target,goals_on_penalty,total_penalties,completed_passings,total_passings,...,shots_on_target_mean,goals_on_penalty_mean,total_penalties_mean,completed_passings_mean,total_passings_mean,corners_mean,fouls_mean,yellow_cards_mean,red_cards_mean,percentage_possession_mean
0,37,Spezia,2,P,12.0,3.0,0,0,403.0,522.0,...,3.666667,0.000000,0.000000,361.000000,443.666667,3.333333,13.666667,2.000000,0.333333,45.333333
0,46,Spezia,2,P,10.0,4.0,0,0,293.0,376.0,...,3.333333,0.000000,0.000000,403.666667,499.000000,5.333333,11.333333,1.333333,0.333333,50.000000
0,52,Spezia,1,P,8.0,2.0,0,0,242.0,319.0,...,4.000000,0.000000,0.000000,376.000000,472.666667,6.333333,13.000000,1.333333,0.000000,50.666667
0,66,Spezia,0,V,12.0,4.0,0,0,444.0,525.0,...,3.000000,0.000000,0.000000,312.666667,405.666667,5.666667,13.000000,1.666667,0.000000,44.000000
0,70,Spezia,2,V,18.0,5.0,0,0,384.0,505.0,...,3.333333,0.000000,0.000000,326.333333,406.666667,4.000000,13.333333,1.666667,0.333333,44.000000
0,80,Spezia,1,V,17.0,7.0,0,0,571.0,700.0,...,3.666667,0.000000,0.000000,356.666667,449.666667,5.666667,13.666667,1.666667,0.333333,50.000000
0,92,Spezia,1,N,13.0,3.0,0,0,319.0,436.0,...,5.333333,0.000000,0.000000,466.333333,576.666667,7.666667,12.333333,1.333333,0.333333,61.333333
0,103,Spezia,0,V,3.0,0.0,0,0,334.0,403.0,...,5.000000,0.000000,0.000000,424.666667,547.000000,7.000000,13.333333,1.666667,0.000000,58.666667
0,113,Spezia,1,V,7.0,3.0,0,0,255.0,360.0,...,3.333333,0.000000,0.000000,408.000000,513.000000,5.000000,15.666667,2.000000,0.000000,54.000000
0,121,Spezia,2,V,10.0,5.0,0,0,417.0,519.0,...,2.000000,0.000000,0.000000,302.666667,399.666667,3.000000,18.000000,2.666667,0.000000,44.333333


,id,team,goals,result,total_shots,shots_on_target,goals_on_penalty,total_penalties,completed_passings,total_passings,...,shots_on_target_mean,goals_on_penalty_mean,total_penalties_mean,completed_passings_mean,total_passings_mean,corners_mean,fouls_mean,yellow_cards_mean,red_cards_mean,percentage_possession_mean
0,30,Torino,1,P,18.0,3.0,0,0,361.0,445.0,...,4.333333,0.000000,0.000000,331.000000,427.333333,5.000000,19.333333,2.333333,0.000000,49.333333
0,47,Torino,1,N,16.0,5.0,0,0,313.0,416.0,...,3.000000,0.000000,0.000000,350.000000,438.000000,4.333333,21.666667,3.000000,0.000000,49.666667
0,59,Torino,1,N,8.0,3.0,0,0,417.0,521.0,...,4.333333,0.000000,0.000000,375.666667,468.333333,5.333333,18.666667,2.333333,0.000000,51.000000
0,63,Torino,0,P,13.0,2.0,0,0,347.0,452.0,...,3.666667,0.000000,0.000000,363.666667,460.666667,3.333333,21.666667,3.666667,0.333333,49.666667
0,73,Torino,0,V,11.0,3.0,0,0,306.0,418.0,...,3.333333,0.000000,0.000000,359.000000,463.000000,3.666667,21.333333,3.333333,0.333333,49.666667
0,81,Torino,3,V,17.0,6.0,0,0,377.0,491.0,...,2.666667,0.000000,0.000000,356.666667,463.666667,3.000000,22.666667,3.666667,0.333333,51.000000
0,91,Torino,0,V,13.0,2.0,0,0,415.0,519.0,...,3.666667,0.000000,0.000000,343.333333,453.666667,3.333333,22.333333,3.000000,0.000000,51.666667
0,100,Torino,3,V,16.0,5.0,0,0,515.0,620.0,...,3.666667,0.000000,0.000000,366.000000,476.000000,3.666667,22.000000,3.000000,0.000000,54.333333
0,113,Torino,0,V,10.0,1.0,0,0,420.0,522.0,...,4.333333,0.000000,0.000000,435.666667,543.333333,3.666667,20.333333,2.333333,0.000000,59.000000
0,128,Torino,2,V,14.0,3.0,0,0,367.0,479.0,...,2.666667,0.000000,0.000000,450.000000,553.666667,4.000000,20.333333,2.666667,0.000000,59.333333


,id,team,goals,result,total_shots,shots_on_target,goals_on_penalty,total_penalties,completed_passings,total_passings,...,shots_on_target_mean,goals_on_penalty_mean,total_penalties_mean,completed_passings_mean,total_passings_mean,corners_mean,fouls_mean,yellow_cards_mean,red_cards_mean,percentage_possession_mean
0,39,Udinese,0,P,9.0,1.0,0,0,346.0,403.0,...,6.333333,0.333333,0.333333,354.666667,434.000000,5.000000,15.000000,1.666667,0.000000,44.333333
0,49,Udinese,0,V,8.0,2.0,0,0,421.0,511.0,...,5.000000,0.000000,0.000000,349.000000,424.000000,4.666667,14.333333,2.000000,0.000000,42.000000
0,57,Udinese,0,P,12.0,5.0,0,0,423.0,506.0,...,2.333333,0.000000,0.000000,366.333333,439.666667,4.333333,12.666667,1.666667,0.000000,43.000000
0,69,Udinese,3,N,18.0,7.0,0,0,294.0,382.0,...,2.666667,0.000000,0.000000,396.666667,473.333333,4.000000,12.333333,1.666667,0.000000,43.666667
0,78,Udinese,1,N,10.0,2.0,0,0,295.0,370.0,...,4.666667,0.000000,0.000000,379.333333,466.333333,6.000000,14.666667,2.666667,0.000000,46.666667
0,88,Udinese,1,N,9.0,3.0,0,0,239.0,333.0,...,4.666667,0.000000,0.000000,337.333333,419.333333,4.666667,17.333333,3.333333,0.333333,44.333333
0,93,Udinese,1,N,16.0,5.0,0,0,202.0,305.0,...,4.000000,0.000000,0.000000,276.000000,361.666667,4.333333,17.333333,3.333333,0.333333,40.000000
0,106,Udinese,0,V,9.0,5.0,0,0,267.0,350.0,...,3.333333,0.000000,0.000000,245.333333,336.000000,4.000000,16.000000,2.000000,0.333333,36.000000
0,114,Udinese,3,V,13.0,2.0,0,0,285.0,378.0,...,4.333333,0.000000,0.000000,236.000000,329.333333,4.666667,16.000000,1.666667,0.000000,33.666667
0,128,Udinese,1,V,18.0,7.0,0,0,351.0,461.0,...,4.000000,0.000000,0.000000,251.333333,344.333333,4.000000,15.000000,2.000000,0.333333,35.333333


,id,team,goals,result,total_shots,shots_on_target,goals_on_penalty,total_penalties,completed_passings,total_passings,...,shots_on_target_mean,goals_on_penalty_mean,total_penalties_mean,completed_passings_mean,total_passings_mean,corners_mean,fouls_mean,yellow_cards_mean,red_cards_mean,percentage_possession_mean
0,37,Venezia,1,P,14.0,3.0,0,0,320.0,425.0,...,3.333333,0.000000,0.000000,355.666667,428.333333,2.000000,19.000000,5.000000,0.000000,43.333333
0,45,Venezia,0,V,8.0,2.0,0,0,257.0,340.0,...,3.000000,0.000000,0.000000,330.666667,418.666667,2.333333,16.333333,4.000000,0.000000,43.666667
0,59,Venezia,1,N,6.0,0.0,1,1,300.0,404.0,...,2.666667,0.000000,0.000000,258.666667,346.666667,3.000000,18.000000,3.666667,0.000000,36.333333
0,60,Venezia,1,N,12.0,4.0,0,0,455.0,548.0,...,1.666667,0.333333,0.333333,292.333333,389.666667,3.666667,12.666667,2.333333,0.000000,40.666667
0,79,Venezia,1,V,4.0,2.0,0,0,294.0,396.0,...,2.000000,0.333333,0.333333,337.333333,430.666667,4.000000,14.333333,2.666667,0.000000,45.000000
0,83,Venezia,1,V,7.0,1.0,0,0,304.0,388.0,...,2.000000,0.333333,0.333333,349.666667,449.333333,3.333333,15.000000,2.666667,0.000000,47.000000
0,90,Venezia,1,P,8.0,4.0,0,0,347.0,441.0,...,2.333333,0.000000,0.000000,351.000000,444.000000,3.333333,17.666667,3.000000,0.000000,44.666667
0,104,Venezia,0,N,6.0,2.0,0,0,235.0,328.0,...,2.333333,0.000000,0.000000,315.000000,408.333333,3.333333,17.333333,2.333333,0.333333,40.333333
0,117,Venezia,3,V,12.0,6.0,1,1,269.0,348.0,...,2.333333,0.000000,0.000000,295.333333,385.666667,4.000000,17.333333,2.666667,0.333333,40.000000
0,126,Venezia,1,P,8.0,6.0,0,0,282.0,353.0,...,4.000000,0.333333,0.333333,283.666667,372.333333,4.000000,16.000000,2.666667,0.333333,40.666667


Dovrei fare la media per 3 partite di tutti i dati, così poi li vado ad aggiungere al set del match